**Author: Christina Alexandra (christina.alexandra@gdplabs.id)**


**Reviewers:**



## References
NONE

# Summary
This notebook is used to run and evaluate GLChat to measure how accurately your selected chatbot responds. The evaluation runs automatically in Google Colab using `evaluate_glchat` function in `glchat-sdk[evals]` and will send the report to Langfuse.

In [ ]:
from google.colab import drive, userdata

import os, subprocess, getpass, tempfile
from dotenv import dotenv_values

## 🚦 Start here — pick your guide
**Please read one of these before running anything:**
- **Simple guide without any customization (recommended for non-engineers):** <a href="https://gdplabs.gitbook.io/sdk/tutorials/evaluation/evaluate-glchat-simple-guide-for-non-engineers" target="_blank">Evaluate GLChat - Simple Guide for Non-Engineers</a>
- **Guide for more customization:** <a href="https://gdplabs.gitbook.io/sdk/tutorials/evaluation/evaluate-glchat-google-colab-guide" target="_blank">Evaluate GLChat - Google Colab Guide</a>

In [ ]:
# @title Dataset & Chatbot Configuration
google_sheets_id = "15oQq2HOM02qP3_ZLm4AStB7fqAhoGlshhkOsOFXCSK8" # @param {"type":"string","placeholder":"e.g. \"abc123ldk_fgght5fdfghht\""}
worksheet_name = "glchat_test_data" # @param {"type":"string","placeholder":"e.g. \"data_test_gloria\""}
chatbot_id = "" # @param {"type":"string","placeholder":"Default value if not set: \"general-purpose\""}
model_name = "" # @param {"type":"string","placeholder":"Default value if not set: \"GPT 5 Mini\""}
user_id = "" # @param {"type":"string","placeholder":"e.g. \"tester_eval1@glair.ai\""}
attachments_gdrive_folder_id = "1u_KP_qaqzfkmBVHFQw76mv-IsI5h8-BY" # @param {"type":"string","placeholder":"Default value if not set: \"1sY7a7yZiAfMlM0ozXlEnzb4EbN84sl2N\""}
attachments_google_client_email = "" # @param {"type":"string","placeholder":"e.g. \"abc.iam.gserviceaccount.com\""}
attachments_google_private_key = "" # @param {"type":"string","placeholder":"e.g. \"abc123...\""}


# 🔑 Login GCloud & Install the Module


In [ ]:
# 1) gcloud user login (click the link and get the token in the browser; paste it below).
!gcloud --version
!gcloud auth login --quiet

In [ ]:
# 2) Mint a token and install the glchat-sdk[evals]
import urllib.parse

token = subprocess.check_output(["gcloud","auth","print-access-token"], text=True).strip()
extra = f"https://oauth2accesstoken:{urllib.parse.quote(token, safe='')}@glsdk.gdplabs.id/gen-ai/simple"
env = os.environ.copy(); env["PIP_EXTRA_INDEX_URL"] = extra
subprocess.run(["python","-m","pip","install","--verbose","glchat-sdk[evals]"], check=True, env=env)
del token, extra, env
print("✅ glchat-sdk[evals] installed via user token.")

# 🔒 Load encrypted secrets from Drive (.env.gpg)

In [ ]:
print("⏳ Mounting Drive…")
drive.mount("/content/drive", force_remount=True)
print("✅ Mounted at /content/drive")

# Check if env path exists
GPG_PATH = "/content/drive/Shareddrives/evaluate_glchat/secrets/.env.gpg"
print("🔎 Checking:", GPG_PATH)
print("📂 Exists?", os.path.exists(GPG_PATH))

In [ ]:
print("⏳ Installing gnupg (if needed)…")
!apt-get update -qq
!apt-get install -y gnupg

print("⏳ Installing python-dotenv…")
%pip install -q python-dotenv
print("✅ Dependencies ready")

In [ ]:
# 1) Enter the passphrase
passphrase = getpass.getpass("🔑 Enter passphrase for secrets: ")

# 2) Decrypt to a temp file (no shell, loopback pinentry)
print("🔐 Decrypting…")
tmp = tempfile.NamedTemporaryFile(prefix=".env_", delete=False)
tmp_path = tmp.name
tmp.close()

proc = subprocess.run(
    ["gpg", "--quiet", "--batch", "--yes",
     "--pinentry-mode", "loopback",
     "--passphrase-fd", "0",
     "--output", tmp_path,
     "--decrypt", GPG_PATH],
    input=passphrase.encode("utf-8"),
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)
# delete passphrase reference
del passphrase

if proc.returncode != 0:
    print("❌ GPG stderr:\n", proc.stderr.decode("utf-8", errors="ignore"))
    raise RuntimeError("GPG decryption failed. Wrong passphrase or unreadable file.")

print("📄 Decrypted to:", tmp_path)

# 3) Load env vars
pairs = dotenv_values(tmp_path)
loaded = []
for k, v in pairs.items():
    if v is not None:
        os.environ[k] = v
        loaded.append(k)

# 4) Cleanup
try: os.remove(tmp_path)
except OSError: pass

print(f"✅ Loaded {len(loaded)} secret(s): {', '.join(loaded)}")
print("   (Values kept in memory only.)")


In [ ]:
# @title Override environment variables from Colab Secrets if needed
from glchat_sdk.evals.constant import REQUIRED_ENV_VARS

def get_colab_secret(name: str):
    try:
        return userdata.get(name)  # raises if missing or not granted
    except userdata.SecretNotFoundError:
        return None
    except userdata.NotebookAccessError:
        # user hasn't granted this notebook access to their secrets
        return None

for key in REQUIRED_ENV_VARS:
    val = get_colab_secret(key)
    if val:
        print(f"Environment variable for {key} is updated...")
        os.environ[key] = val

# 📚 Start Evaluation

In [ ]:
# @title Verify the Environment Variables
from langfuse import get_client

try:
  # Initialize Langfuse client
  langfuse = get_client()

  # Verify Langfuse connection
  if langfuse.auth_check():
      print("Langfuse client is authenticated and ready!")
  else:
      print("Authentication failed. Please check your credentials and host.")
except Exception as err:
  print(f"There is a problem with Langfuse: {err}")

# Verify if GLChat env vars exist and has a non-empty value
GLCHAT_BASE_URL = os.getenv("GLCHAT_BASE_URL")
GLCHAT_API_KEY = os.getenv("GLCHAT_API_KEY")
if GLCHAT_BASE_URL and GLCHAT_API_KEY:
    print("GLCHAT_BASE_URL and GLCHAT_API_KEY variables are available!")
else:
    print("GLCHAT_BASE_URL and GLCHAT_API_KEY variables are not set or empty.")

In [ ]:
# @title Set default values if not specified
chatbot_id = "general-purpose" if not chatbot_id else chatbot_id
model_name = "GPT 5 Mini" if not model_name else model_name
user_id = "tester_eval1@glair.ai" if not user_id else user_id
attachments_gdrive_folder_id = "1sY7a7yZiAfMlM0ozXlEnzb4EbN84sl2N" if not attachments_gdrive_folder_id else attachments_gdrive_folder_id
attachments_google_client_email = os.getenv("GOOGLE_SHEETS_CLIENT_EMAIL") if not attachments_google_client_email else attachments_google_client_email
attachments_google_private_key = os.getenv("GOOGLE_SHEETS_PRIVATE_KEY") if not attachments_google_private_key else attachments_google_private_key

In [ ]:
# @title Perform end-to-end Evaluation
import asyncio
import json
import os

from langfuse import get_client

from gllm_evals.evaluator.geval_generation_evaluator import GEvalGenerationEvaluator
from gllm_evals.experiment_tracker.langfuse_experiment_tracker import LangfuseExperimentTracker
from gllm_evals.types import GoogleDriveAttachmentConfig

from glchat_sdk.evals import GLChatConfig, evaluate_glchat


async def main():
    """The main function to run the evaluation."""

    # The GLChat configuration
    config = GLChatConfig(
        base_url=GLCHAT_BASE_URL,
        api_key=GLCHAT_API_KEY,
        chatbot_id=chatbot_id,
        username=user_id,
        model_name=model_name,  # the bigger priority is the one in the dataset columns
    )

    # The attachments config
    attachments_config = GoogleDriveAttachmentConfig(
        client_email=os.getenv("GOOGLE_SHEETS_CLIENT_EMAIL"),
        private_key=os.getenv("GOOGLE_SHEETS_PRIVATE_KEY"),
        folder_id=attachments_gdrive_folder_id,
    )

    geval_evaluator = GEvalGenerationEvaluator(
        model="google/gemini-2.5-pro",
        model_credentials=os.getenv("GOOGLE_API_KEY"),
    )
    geval_evaluator.name = "generation"

    # Call the `evaluate_glchat`
    results = await evaluate_glchat(
        data=f"gs/{worksheet_name}",
        evaluators=[geval_evaluator],
        experiment_tracker=LangfuseExperimentTracker(
            langfuse_client=get_client(),
            project_name="glchat_beta",
        ),
        config=config,
        attachments_config=attachments_config,
        sheet_id=google_sheets_id,
        client_email=os.getenv("GOOGLE_SHEETS_CLIENT_EMAIL"),
        private_key=os.getenv("GOOGLE_SHEETS_PRIVATE_KEY"),
    )
    print(results)


if __name__ == "__main__":
    asyncio.run(main())